## Note:
- This code has to be run under **python 2.5+**
- This part is to get the sentiment score through package ***pattern, senticnet, and vader.***
- In order to run this code, the env needs to install all the package listed above.

In [2]:
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import re
#from pattern.en import sentiment
import requests

In [3]:
with open ('reviews_update.csv', 'r') as review:
    df = pd.read_csv(review, encoding = 'utf8')
    
col_name = ['index','movie', 'review_title', 'rate', 'body', 'helpful']
df.columns = col_name

In [4]:
for i in range(len(df['rate'])):
    score = df['rate'][i].split()[0]
    df.loc[i, 'rate']= score[:score.find('/')]

In [4]:
def decide_using_pattern(sentence_list):
    '''give sentiment score based on the senticnet package
    and will reture two set of score, one for positive words
    and one for negative words
    return a list [pos_score, neg_score]'''
    positive_sentiment = []
    negative_sentiment = []
    ref = []
    for s in sentence_list:
        try:
            polarity = sentiment(s)
            # the first number return by pattern is the sentiment
            sentence_intense = polarity[0]
            if sentence_intense>=0:
                positive_sentiment.append(str(sentence_intense))
            else:
                negative_sentiment.append(str(sentence_intense))
        except:
            next
    
    if positive_sentiment == ref:
        pos_score = [0]
    if negative_sentiment == ref:
        neg_score = [0]
    score = [positive_sentiment, negative_sentiment]
    return score

In [5]:
def splitParagraph_to_Sentences(paragraph):
    ''' break a paragraph into sentences
        and return a list '''
    
    #   regular expressions are easiest (and fastest)
    sentenceEnders = re.compile('[.!?]')
    sentenceList = sentenceEnders.split(paragraph)
    for index, line in enumerate(sentenceList):
        if line:
            sentenceList[index] = line.strip()
        else:
            sentenceList.pop(index)
    return sentenceList

In [6]:
def pattern_sentiment(a_df):
    for i in range(len(a_df)):
        score_senti = None
        content = a_df.loc[i, 'body']
        content_list = splitParagraph_to_Sentences(content)
        score_senti = decide_using_pattern(content_list)
        a_df.loc[i, 'pattern_pos'] = ';'.join(score_senti[0])
        a_df.loc[i, 'pattern_neg'] = ';'.join(score_senti[1])
        
    return a_df

In [7]:
%%time
df_pattern = pattern_sentiment(df) 

CPU times: user 3min 17s, sys: 2.42 s, total: 3min 19s
Wall time: 3min 19s


In [2]:
from senticnet.senticnet import SenticNet

In [3]:
sn = SenticNet()

In [4]:
import spacy

nlp = spacy.load('en')

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.corpus import stopwords

In [12]:
my_stop_word = stopwords.words('english')

In [13]:
def noun_adj_verb_filter(sentences, stop_words=None):
    """
    Only keep nouns, adjectives, adverb and verbs in the sentence.
    """
    preprocessed_sentences = []
    #keep_types = {'NOUN', 'PROPN', 'ADJ', 'VERB', 'ADV'}
    keep_types = {'ADJ', 'VERB', 'ADV'}
    
    
    for sentence in sentences:
        doc = nlp(unicode(sentence))
        keep_tokens_string = ' '.join([t.lemma_ for t in doc if t.pos_ in keep_types and t.lemma_ not in stop_words])
        preprocessed_sentences.append(keep_tokens_string)
        
    return preprocessed_sentences

In [14]:
def decide_sentiment(text):
    '''give sentiment score based on the senticnet package
    and will reture two set of score, one for positive words
    and one for negative words
    return a list [pos_score, neg_score]'''
    positive_sentiment = []
    negative_sentiment = []
    doc = nlp(unicode(text))
    sn = SenticNet()
    for t in doc:
        try:
            polarity = sn.polarity_intense(t.text)
            word_intense = float(polarity)
            if word_intense>=0:
                positive_sentiment.append(str(word_intense))
            else:
                negative_sentiment.append(str(word_intense))
        except:
            next

    score = [positive_sentiment, negative_sentiment]
    return score

In [15]:
def senticnet_sentiment(a_df):
    # using the senticnet method to give two score, a positive score count only on postive words
    # and a negative score count only on negtive words
    for i in range(len(a_df)):
        score_senti = None
        content = a_df.loc[i, 'body']
        content_list = [content]
        #print content_list
        pre_process = noun_adj_verb_filter(content_list, stop_words=my_stop_word)
        #print pre_process
        score_senti = decide_sentiment(pre_process[0])
        a_df.loc[i, 'senticnet_pos'] = ';'.join(score_senti[0])
        a_df.loc[i, 'senticnet_neg'] = ';'.join(score_senti[1])
        
    return a_df

In [16]:
%%time
a = senticnet_sentiment(df_pattern)

CPU times: user 54min 16s, sys: 8min 5s, total: 1h 2min 22s
Wall time: 40min 14s


In [17]:
s_df = a

In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/Users/xusibocn/anaconda/envs/project_py2/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [8]:
analyzer = SentimentIntensityAnalyzer()

In [21]:
def vader_sentiment(a_df):
    '''get sentiment score from vader'''
    for i in range(len(a_df)):
        score_senti = None
        content = a_df.loc[i, 'body']
        content_list = splitParagraph_to_Sentences(content)
        pos_vader = []
        neg_vader = []
        for sentence in content_list:
            sentiment = analyzer.polarity_scores(sentence)
            positive = str(sentiment['pos'])
            pos_vader.append(positive)
            negative = str(-sentiment['neg'])
            neg_vader.append(negative)

        a_df.loc[i, 'vader_pos'] = ';'.join(pos_vader)
        a_df.loc[i, 'vader_neg'] = ';'.join(neg_vader)
        
    return a_df

In [22]:
v_df = vader_sentiment(s_df)

In [23]:
def remove_zero(a_string):
    a_list = a_string.split(';')
    ref = []
    for i in a_list:
        if not i == '':
            if i == '0.0':
                a_list.remove('0.0')
            elif i == '-0.0':
                a_list.remove('-0.0')
            output = a_list
        else:
            output = ['0']
    if output == ref:
        output = ['0']
    return output

In [24]:
def listmean (input_list):
    for i, v in enumerate(input_list):
        if v:
            input_list[i] = float(v)
        else:
            input_list[i] = 0
    new_list = input_list
    listmean = sum(new_list)/len(new_list)
    
    return listmean

In [25]:
df = v_df

In [26]:
review_sentiment = []
for i in range(len(df.index)):
    index = df.loc[i, 'index']
    movie = df.loc[i, 'movie']
    review = df.loc[i, 'body']
    rate = df.loc[i, 'rate']
    vader_pos = df.loc[i,'vader_pos']
    vader_neg = df.loc[i,'vader_neg']
    senticnet_pos = df.loc[i,'senticnet_pos']
    senticnet_neg = df.loc[i,'senticnet_neg']
    vader_pos_mean = listmean(remove_zero(vader_pos))
    vader_neg_mean = listmean(remove_zero(vader_neg))
    senticnet_pos_mean = listmean(remove_zero(senticnet_pos))
    senticnet_neg_mean = listmean(remove_zero(senticnet_neg))
    pattern_pos = df.loc[i,'pattern_pos']
    pattern_neg = df.loc[i,'pattern_neg']
    pattern_pos_mean = listmean(remove_zero(pattern_pos))
    pattern_neg_mean = listmean(remove_zero(pattern_neg))
    df.loc[i,'pattern_pos'] = pattern_pos_mean
    df.loc[i,'pattern_neg'] = pattern_neg_mean
    review_info = [index, movie, review, rate, vader_pos_mean, vader_neg_mean, senticnet_pos_mean, senticnet_neg_mean, pattern_pos_mean, pattern_neg_mean]
    review_sentiment.append(review_info)

In [27]:
df_three_sentiment = pd.DataFrame(review_sentiment, columns=['index', 'movie', 'body','rate', 
                                         'vader_pos', 'vader_neg', 'senticnet_pos',
                                        'senticnet_neg', 'pattern_pos', 'pattern_neg'])

In [29]:
df_three_sentiment.to_csv('review_sentiment_score_updated.csv', index='index', encoding = 'utf8' )

In [28]:
df_three_sentiment

,index,movie,body,rate,vader_pos,vader_neg,senticnet_pos,senticnet_neg,pattern_pos,pattern_neg
0,0,10 Years,"Before I watched this movie, I read some revie...",10,0.237875,-0.079000,0.575030,-0.532500,0.355022,-0.500000
1,1,10 Years,"In his directorial debut ""Ten Year"", Jamie Lin...",7,0.223286,-0.051250,0.452878,-0.453750,0.203770,-0.116250
2,2,10 Years,'10 Years' is all about reunion of high school...,7,0.295000,-0.100000,0.502971,-0.630000,0.302272,-0.093333
3,3,10 Years,¨Now the party can start!¨The film is about a ...,7,0.226742,-0.043609,0.548691,-0.677368,0.316566,-0.115972
4,4,10 Years,Definitely worth seeing. I thoroughly enjoyed ...,9,0.388500,-0.013500,0.416000,-0.560000,0.260781,0.000000
5,5,10 Years,What you can expect from 10 Years is a predict...,6,0.228750,-0.012333,0.421511,-0.838889,0.123810,-0.090417
6,6,10 Years,High school reunions can be equal parts helpfu...,7,0.179714,-0.090364,0.493229,-0.683103,0.204139,-0.114635
7,7,10 Years,This movie was fun but parts where very predic...,6,0.151750,-0.058286,0.580440,-0.818750,0.109494,0.000000
8,8,10 Years,"In ""Grosse Point Blank,"" John Cusack, a hit-ma...",5,0.105571,-0.103875,0.346324,-0.842667,0.162778,-0.111979
9,9,10 Years,I'll make this short. It was boring. Though it...,3,0.168714,-0.135429,0.520625,-0.651818,0.245908,-0.541667


## demo of three sentiment score

- vader

In [10]:
text = 'I love class reunions.'
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(text)

{'compound': 0.6369, 'neg': 0.0, 'neu': 0.323, 'pos': 0.677}

- Pattern

In [12]:
sentiment('love')

(0.5, 0.6)

In [13]:
sentiment('hate')

(-0.8, 0.9)

- SenticNet

In [16]:
sn = SenticNet()
sn.polarity_intense('love')

'0.83'